In [ ]:
# 디렉토리 구조
import os
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
list_files('../')

# 현재 디렉토리 출력
import os.path
folder=os.getcwd()
print ('current directory :%s' % folder)

for filename in os.listdir(folder):
    ext=filename.split('.')[-1]
    if ext == 'exe':
        print(filename)

In [ ]:
'''
[처음 보는 라이브러리 정리]

(1) easydict
-> key값 대신 (obj).attribute의 도트 표기법으로 접근할 수 있는 딕셔너리를 제공하는 라이브러리
-> ipynb 에서 사용이 불가한 argparse 대신 easydict을 사용해 인자를 삽입/수정할 수 있다. 

(2) natsort
-> 텍스트로 된 숫자(파일명)을 정렬하기 위한 라이브러리. 
-> 지정된 경로의 파일명을 정렬해서 리스트 형식으로 반환.

(3) ptflops
-> (pytorch로 생성된?) 딥러닝 모델의 연산량(flops)를 계산해주기 위한 라이브러리 

(4) StratifiedKFold /  StratifiedGroupKFold
-> k-fold cross validation(교차검증): 
교차 검증법은 overfitting을 방지하기 위한 기법 중 하나로 test 데이터와 별개의 validation set, k개의 학습 데이터 fold를 만들어 바꾸어 가면서 학습을 진행하는 것이다.
-> stratified k-fold는 y 값(label 데이터)의 분포에 따라 train/validation 데이터를 나눈다. 각 fold에 클래스 비율을 동일하게 갖도록 설정함으로써 데이터가 몰리는 것을 방지한다. 

(5) timm
-> py'T'orch 'IM'age 'M'odels 이다.
-> 컴퓨터 비전 분야의 SOTA모델(pretrained on ImageNet data), 레이어, optim, dataloader, augmentation등의 유용한 툴을 제공하는 라이브러리이다. 

(6) glob
-> 파이썬 프로그램을 작성할 때, 특정한 패턴이나 확장자를 가진 파일들의 경로나 이름이 필요할 때가 있다.
glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다. 
-> 단, 조건에 정규식을 사용할 수 없으며 엑셀 등에서도 사용할 수 있는 '*'와 '?'같은 와일드카드만을 지원한다.

'''

In [1]:
# Libraray import
import os
import cv2 # openCV
import json
import time
import random
import logging
import easydict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from pathlib import Path
from natsort import natsorted
from os.path import join as opj
from ptflops import get_model_complexity_info
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from PIL import Image

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# argparse 가 아닌 easydict을 이용해서 input arguments(hyper parameters)작성

args = easydict.EasyDict(
    {'exp_num':'0',
     'experiment':'Base',
     'tag':'Default',

     # Path settings
     'data_path':'../data',
     'fold':4,
     'Kfold':5,
     'model_path':'results/',

     # Model parameter settings
     'encoder_name':'regnety_040',
     'drop_path_rate':0.2,
     
     # Training parameter settings
     ## Base Parameter
     'img_size':288,
     'batch_size':16,
     'epochs':60,
     'optimizer':'Lamb',
     'initial_lr':5e-6,
     'weight_decay':1e-3,

     ## Augmentation
     'aug_ver':2,
     'flipaug_ratio':0.3,
     'margin':50,
     'random_margin':True,

     ## Scheduler
     'scheduler':'cycle',
     'warm_epoch':5,
     ### Cosine Annealing
     'min_lr':5e-6,
     'tmax':145,
     ### OnecycleLR
     'max_lr':1e-3,

     ## etc.
     'patience':50,
     'clipping':None,

     # Hardware settings
     'amp':True,
     'multi_gpu':False,
     'logging':False,
     'num_workers':4,
     'seed':42
    })

In [ ]:
'''
[Image data augmentation]
"A survey on Image Data Augmentation for Deep Learning" 
(https://journalofbigdata.springeropen.com/track/pdf/10.1186/s40537-019-0197-0.pdf)
-> data augmentation은 overfitting 상황에서 variance를 낮추기 위한 Regularization 기법의 일종으로 원본 train data에 여러 가지 변환을 처리해 데이터의 양을 늘리는 작업이다.   

(1) image random crop
-> 아래 crop_image 함수가 구현하고 있는 random crop data augmentation은 이미지 데이터의 일부 영역에서 추출한 feature만을 가지고도 라벨 판별이 가능하도록 하기 위해 사용한다. 
-> 좌표를 가리키는 array point를 기준으로 각 x,y좌표보다 margin만큼 더 크고, 작은 영역을 crop한다. 

(2) image filp
-> 이미지를 filp해 training data를 늘리는 방법(좌우)(상하 반전은 잘 사용 X)

(3) image rotation
-> 이미지를 랜덤한 방향으로 랜덤한 각도만큼 회전시켜 training set으로 사용. task에 따라 bounding box 재정의가 필요하다.(object detection)
-> 더 많은 라벨을 가진 데이터를 수집할 수 없을 때 사용하는 증강 방법

'''

In [ ]:
# keypoint를 기준으로 이미지를 crop하기 위한 함수 정의
# train과 test시 해당 함수가 적용된 crop이미지가 inputs으로 들어가게 됩니다.
def crop_image(imges, point, margin=100):
    image = np.array(Image.open(imges).convert('RGB'))
    point = point['data'] 
    max_point = np.max(np.array(point), axis=0).astype(int) + margin
    min_point = np.min(np.array(point), axis=0).astype(int) - margin
    max_point = max_point[:-1] # remove Z order
    min_point = min_point[:-1] # remove Z order

    max_x, max_y = max_point
    min_x, min_y = min_point
    max_y += margin  # 손목까지
    
    # 데이터 포인트의 크기가 원 이미지를 넘어서는 경우를 방지
    max_x = max_x if max_x < 1920 else 1920
    max_y = max_y if max_y < 1080 else 1080
    min_x = min_x if min_x > 0 else 0
    min_y = min_y if min_y > 0 else 0
    
    crop_image = image[min_y:max_y, min_x:max_x]

    return crop_image

In [ ]:
'''
[데이터 형식]
 # 다운받은 데이터는 train, test 폴더가 있다.
train data 폴더 안에는 정수 명의 폴더목록이 있으며 그 안에는 같은 손동작 사진이 여러장과 json파일이 들어있음.
jason 파일에는 "action", "actor","id","annotations" 속성이 들어있다. 
"action"은 len 2 짜리 리스트 (eg.[175, "\uc190 \uc548\uacbd"]) -> 무슨 의미? 손동작의 label
"actor"는 배우에 대한 정보
"annotations"은 "image_id"와 "data"키를 가지는 딕셔너리, data는 각 png이미지 파일의 poision-> [이미지의 id, label, key_point 정보.]
data/
        df_train.csv
        df_train_add.csv
        hand_gesture_pose.csv
        sample_submission.csv
        test/
            649/ 
                0.png, 1.png, 2.png, 3.png, 4.png, 5.png ,6.png, 649.json
                . 
                . 
                .
            865/
                0.png, 1.png, 2.png, 3.png, 4.png, 5.png ,6.png, 649.json
            
        train/
            0/ 
                0.png, 1.png, 2.png, 3.png, 4.png, 5.png ,6.png, 649.json
                . 
                . 
                .
            653/
                0.png, 1.png, 2.png, 3.png, 4.png, 5.png ,6.png, 649.json
'''

In [ ]:
# dataloader에서 사용할 dataframe 만들기

train_path = '../data/train'
train_folders = natsorted(glob(train_path + '/*')) # train 폴더 안의 데이터 폴더명(정수) 정렬

answers = []
for train_folder in train_folders:
    json_path = glob(train_folder + '/*.json')[0]
    js = json.load(open(json_path))
    cat = js.get('action')[0]
    cat_name = js.get('action')[1]
    
    images_list = glob(train_folder + '/*.png') # 폴더 안의 png 이미지, 갯수는 다 다름
    for image_name in images_list:
        answers.append([image_name, cat, cat_name]) # 이미지 번호와 정답 label(cat, cat_name)을 answer로 묶가

answers = pd.DataFrame(answers, columns = ['train_path','answer', 'answer_name'])
answers.to_csv('../data/df_train.csv', index=False) # 라벨들을 따로 묶은 것 csv로 저장

# 클래스가 1개뿐인 폴더들 Augmentation해서 이미지 생성후 dataframe재정의
# 새롭게 정의한 dataframe을 학습에 이용시 약간의 성능향상을 확인할 수 있었음.
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

# data_path = '../data'
data_path = 'C:/Users/cse/Desktop/Jupyter/eruron/data' # -> 하위 폴더 아니라 상위 폴더에 저장되서 직접 경로 바꿈 (왜지?)

df_train = pd.read_csv(opj(data_path, 'df_train.csv'))
df_info = pd.read_csv(opj(data_path, 'hand_gesture_pose.csv'))
# 키값 기준으로 두 csv파일의 열 병합 
df_train = df_train.merge(df_info[['pose_id', 'gesture_type', 'hand_type']],
                        how='left', left_on='answer', right_on='pose_id') # how='left' -> df2에 값이 없는 경우 NaN

save_folder = 'train' 
for i in range(649, 649+5): 
    if not os.path.exists(opj(data_path, save_folder, str(i))):  # 645~654 폴더 추가 -> 아래 aug data
        os.makedirs(opj(data_path, save_folder, str(i)))

# flip aug가능한 label : 131, 47 (one sample)
oslabel_fliplabel = [(131,156), (47, 22)] # one sample label, flip label  # -> 한개 아닌데...?
folders = ['649', '650'] # Train 648번 folder에 이은 number 생성
for label, folder in tqdm(zip(oslabel_fliplabel, folders)):
    idx = 0
    os_label, f_label  = label[0], label[1]
    one_sample = df_train[df_train['answer'] == os_label].reset_index(drop=True)
    temp = df_train[df_train['answer'] == f_label].reset_index(drop=True)
    train_folders = natsorted(temp['train_path'].apply(lambda x : x[:-6]).unique())
    for train_folder in (train_folders):
        json_path = glob(train_folder + '/*.json')[0]
        js = json.load(open(json_path))
        keypoints = js['annotations']
        images_list = natsorted(glob(train_folder + '/*.png'))
        for _, (point, image_name) in enumerate(zip(keypoints, images_list)):
            print("point: ",point)
            croped_image = crop_image(image_name, point, margin=50)
            flip_img = cv2.flip(croped_image, 1)
            save_path = opj(data_path, save_folder, folder, f'{idx}.png')
            idx += 1
            cv2.imwrite(save_path, flip_img)
            df_train.loc[len(df_train)] = [save_path] + one_sample.iloc[0][1:].values.tolist()

def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h)) 
    return img

oslabel = [92, 188, 145]
folder = ['651', '652', '653']
for label, folder in tqdm(zip(oslabel, folder)):
    idx = 0
    one_sample = df_train[df_train['answer'] == label].reset_index(drop=True)
    train_folders = natsorted(temp['train_path'].apply(lambda x : x[:-6]).unique())
    for train_folder in (train_folders):
        json_path = glob(train_folder + '/*.json')[0]
        js = json.load(open(json_path))
        keypoints = js['annotations']
        images_list = natsorted(glob(train_folder + '/*.png'))
        for _, (point, image_name) in enumerate(zip(keypoints, images_list)):
            croped_image = crop_image(image_name, point, margin=50)
            aug_img = rotation(croped_image, 30)
            save_path = opj(data_path, save_folder, folder, f'{idx}.png')
            idx += 1
            cv2.imwrite(save_path, aug_img)
            df_train.loc[len(df_train)] = [save_path] + one_sample.iloc[0][1:].values.tolist()

df_train.to_csv('../data/df_train_add.csv', index=False)


In [ ]:
# Train dataset에 475, 543 폴더는 의도하지 않은 나머지 손에 대해서도 Keypoint가 잡히게 됨.
# Json의 Keypoint를 사용하기위해 475,543폴더인 경우 해당 부분 Keypoint 제거
def remove_keypoints(folder_num, points):
    lst = []
    for x,y,z in points:
        cond1 = x<250 and y>800
        cond2 = x>1400 and y<400
        if not (cond1 or cond2):
            lst.append([x,y,z]) 
    # print('Finished removing {} wrong keypoints....'.format(folder_num))
    return lst

In [ ]:
# train dataframe 에 augmentation 적용
class Train_Dataset(Dataset):
    def __init__(self, df, transform=None, df_flip_info=None, flipaug_ratio=0, label_encoder=None, margin=50, random_margin=True):
        self.id = df['train_path'].values
        self.target = df['answer'].values
        self.transform = transform
        self.margin = margin
        self.random_margin = random_margin

        # Flip Augmentation (Change target class)
        if df_flip_info is not None:
            self.use_flip = True
            print('Use Flip Augmentation')
            left = label_encoder.transform(df_flip_info['left'])
            right = label_encoder.transform(df_flip_info['right'])
            left_to_right = dict(zip(left, right))
            right_to_left = dict(zip(right, left))
            
            self.flip_info = left_to_right.copy()
            self.flip_info.update(right_to_left)        
            self.flip_possible_class = list(set(np.concatenate([left, right])))
        self.flipaug_ratio = flipaug_ratio

        print(f'Dataset size:{len(self.id)}')

    def __getitem__(self, idx):
        image = np.array(Image.open(self.id[idx]).convert('RGB'))
        target = self.target[idx]

        # Load Json File
        try:
            image_num = int(Path(self.id[idx]).stem)
            dir = os.path.dirname(self.id[idx])
            folder_num = os.path.basename(dir)
            json_path = opj(dir, folder_num+'.json')
            js = json.load(open(json_path))
            keypoints = js['annotations'][image_num]['data']  # 해당 이미지에 해당하는 Keypoints
        except:  # Augmentation으로 직접 새로 만든 Folder는 Json이 없으므로 바로 Return (미리 손 부분이 Crop된 상태로 저장하였음.)
            image = self.transform(Image.fromarray(image))
            return image, np.array(target)

        if folder_num in ['475', '543']:
            keypoints = remove_keypoints(folder_num, keypoints)

        # Image Crop using keypoints  -> crop_image 왜 안써?
        max_point = np.max(np.array(keypoints), axis=0).astype(int) + self.margin
        min_point = np.min(np.array(keypoints), axis=0).astype(int) - self.margin
        max_point = max_point[:-1] # remove Z order
        min_point = min_point[:-1] # remove Z order

        max_x, max_y = max_point
        min_x, min_y = min_point
        max_y += 100  # 손목부분까지 여유를 주기위해

        # 매 에폭마다 Margin이 조금씩 다르게 들어가므로 한 폴더 내 비슷한 이미지들의 Overfitting을 방지하는 효과를 주기위해 (Only Train Phase)
        if self.random_margin:  
            if random.random() < 0.5:
                max_x += self.margin
            if random.random() < 0.5:
                max_y += self.margin
            if random.random() < 0.5:
                min_x -= self.margin
            if random.random() < 0.5:
                min_y -= self.margin
        else:
            max_x += self.margin
            max_y += self.margin
            min_x -= self.margin
            min_y -= self.margin

        # 데이터 포인트의 크기가 원 이미지를 넘어서는 경우를 방지
        max_x = max_x if max_x < 1920 else 1920
        max_y = max_y if max_y < 1080 else 1080
        min_x = min_x if min_x > 0 else 0
        min_y = min_y if min_y > 0 else 0
        
        image = image[min_y:max_y, min_x:max_x]

        # FlipAug
        if (random.random() < self.flipaug_ratio) and (target in self.flip_possible_class):
            image = np.flip(image, axis=1)  # (H, W, C)에서 width 축 flip
            target = self.flip_info[target]

        image = self.transform(Image.fromarray(image))
        return image, np.array(target)

    def __len__(self):
        return len(self.id)

def get_loader(df, batch_size, shuffle, num_workers, transform, df_flip_info=None, 
                flipaug_ratio=0, label_encoder=None, margin=50, random_margin=True):
    dataset = Train_Dataset(df, transform, df_flip_info=df_flip_info, flipaug_ratio=flipaug_ratio, 
                            label_encoder=label_encoder, margin=margin, random_margin=random_margin)
    # torch.utils.data.DataLoader (iterable generator) 반환
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True,
                                drop_last=False)
    return data_loader

# transform 전처리
def get_train_augmentation(img_size, ver): 
    if ver==1:
        # For Test
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)), # PIL 사이즈 재조정
                transforms.ToTensor(), # np tensor 형변환
                transforms.Normalize(mean=[0.485, 0.456, 0.406], # 정규화
                                     std=[0.229, 0.224, 0.225]),
                ])


    if ver==2:
        # For Train
        transform = transforms.Compose([
                transforms.RandomAffine(20), 
                transforms.RandomPerspective(),
                transforms.ToTensor(),
	            transforms.Resize((img_size, img_size)),
    	        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]),
            ])


    return transform

In [ ]:
'''
[전처리 기법]
perspective transformation(= Homography Matrix = Projective Transformation)
Homograhpy
한 평면을 다른 평면에 투영(Projection) 시켰을 때, 투영된 대응점들 사이에서는 일정한 변환 관계가 성립한다. 이 변환 관계를 Homography라고 한다.
점 매핑을 통해 projection으로 인한 변화를 보정해주는 변환 (-> SIFT같은 기법?)
'''

In [ ]:
# Network
'''
[regnet] 
"Designing Network Design Spaces"
최근 NAS가 많이 사용되고 있지만 이런 방식은 특정 세팅에만 적잡한 single network instance를 만들기 때문에 general하지 않다는 단점이 있다.
->  manual design과 NAS 방식의 각각의 장점을 조합한 새로운 디자인 패러다임
-> manual desig와 같이 네트워크 구조를 설명이 가능하며 심플하고 모든 세팅을 아울러 잘 작동하는 네트워크를 만들 수 있는 general한 design principle을 발견하는 것을 목표로 한다.
NAS와 같이 이러한 목표를 달성하도록 도와주는 semi-automated procedure의 장점을 활용한다.
-> 결국, 전체적으로 하나의 특정 network instance를 만드는 것이 목표가 아닌 network의 population들을 파라미터화 시킨 design space를 디자인 하는 것이 목표이다.
-> stem-body-cell
'''

In [ ]:
class Pose_Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.encoder = timm.create_model(args.encoder_name, pretrained=True,
                                    drop_path_rate=args.drop_path_rate,
                                    ) # timm으로 pretrained 모델 불러오기
        num_head = self.encoder.head.fc.in_features
        self.encoder.head.fc = nn.Linear(num_head, 157)
    
    def forward(self, x):
        return self.encoder(x)

In [ ]:
'''
[learning rate scheduler]
Lr Scheduler는 미리 학습 일정을 정해두고, 그 일정에 따라 학습률을 조정하는 방법이다. 
일반적으로는 warmup이라는 파라미터를 정하고 현재 step이 warmup보다 낮을 경우는 learning rate를 linear하게 증가 시키고, 
warmup 이후에는 각 Lr Scheduler에서 정한 방법대로 learning rate를 update한다.

Constant Lr Scheduler, Exponential Lr Scheduler, King Lr Scheduler, Cosine Lr Scheduler 등이 있다.
'''

In [ ]:
# Warmup Learning rate scheduler
from torch.optim.lr_scheduler import _LRScheduler
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

# Logging - INFO log 기록
def get_root_logger(logger_name='basicsr',
                    log_level=logging.INFO,
                    log_file=None):

    logger = logging.getLogger(logger_name)
    # if the logger has been initialized, just return it
    if logger.hasHandlers():
        return logger

    format_str = '%(asctime)s %(levelname)s: %(message)s'
    logging.basicConfig(format=format_str, level=log_level)

    if log_file is not None:
        file_handler = logging.FileHandler(log_file, 'w')
        file_handler.setFormatter(logging.Formatter(format_str))
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    return logger

class AvgMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.losses = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.losses.append(val)

In [ ]:
class Trainer():
    def __init__(self, args, save_path):
        '''
        args: arguments
        save_path: Model 가중치 저장 경로
        '''
        super(Trainer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Logging
        log_file = os.path.join(save_path, 'log.log')
        self.logger = get_root_logger(logger_name='IR', log_level=logging.INFO, log_file=log_file)
        self.logger.info(args)
        self.logger.info(args.tag)

        # Train, Valid Set load
        ############################################################################
        # df_train = pd.read_csv(opj(args.data_path, 'df_train.csv'))
        df_train = pd.read_csv(opj(args.data_path, 'df_train_add.csv'))
        df_info = pd.read_csv(opj(args.data_path, 'hand_gesture_pose.csv'))

        df_train = df_train.merge(df_info[['pose_id', 'gesture_type', 'hand_type']], \
                                how='left', left_on='answer', right_on='pose_id')

        # 폴더별(Group)로 각 번호 부여
        df_train['groups'] = df_train['train_path'].apply(lambda x:x.split('/')[3])
        df_train.loc[:,:] = natsorted(df_train.values)
        # 노이즈 이미지 제거: 596번은 주먹쥐기 이미지인데 갑자기 손바닥을 펴는 노이즈 이미지가 5장있음 + 0번 폴더에 9번 이미지 역시 잘못된 클래스
        drop_idx = df_train[df_train['groups'].isin(['596'])].index.tolist()[3:8] + [9]  
        df_train = df_train.drop(drop_idx).reset_index(drop=True)  
        le = LabelEncoder()
        df_train['answer'] = le.fit_transform(df_train['answer'])
        
        # Split Fold
        # kf = StratifiedGroupKFold(n_splits=args.Kfold)
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=args.seed)
        for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y=df_train['answer'])):
            df_train.loc[val_idx, 'fold'] = fold
        df_val = df_train[df_train['fold'] == args.fold].reset_index(drop=True)
        df_train = df_train[df_train['fold'] != args.fold].reset_index(drop=True)
        
        # Augmentation
        self.train_transform = get_train_augmentation(img_size=args.img_size, ver=args.aug_ver)
        self.test_transform = get_train_augmentation(img_size=args.img_size, ver=1)
        
        ######################################################################
        # Flip Augmentation을 위한 Mapping dataframe
        df_info = pd.read_csv('../data/hand_gesture_pose.csv')
        df_info = df_info[df_info['hand_type'] != 'both']
        # drop idx, 동일한 약속, gesture_type, hand_type인데 다른 클래스인 경우 존재 -> 약속 1과 2로 이름을 나누어줌.
        df_info.loc[[105, 128], 'pose_name'] = '약속 1'  # idx: (105, 128)
        df_info.loc[[101, 124], 'pose_name'] = '약속 2'  # idx: (101, 124)

        # drop 41 idx, 동일한 약속, my hand, right class가 49와 54로 두 개있어 Mapping df만들 때 문제가 발생하여 미리 49번 클래스 처리
        df_info = df_info.drop(41)

        # Make a mapping dataframe
        df_info = df_info.groupby(['pose_name', 'view_type', 'gesture_type', 'hand_type']).sum().unstack().reset_index().dropna(axis=0)
        df_info['left'] = df_info.pose_id.left.apply(int)
        df_info['right'] = df_info.pose_id.right.apply(int)
        df_flip_info = df_info.drop('pose_id', axis=1).droplevel('hand_type', axis=1).reset_index(drop=True)
        print('Mapping dataframe Length', df_flip_info.shape)
        ######################################################################
        
        # 이 위까지 데이터 읽고 전처리 적용
        
        # TrainLoader
        self.train_loader = get_loader(df_train, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, transform=self.train_transform, 
                                       df_flip_info=df_flip_info, flipaug_ratio=args.flipaug_ratio, label_encoder=le, margin=args.margin, random_margin=args.random_margin)
        self.val_loader = get_loader(df_val, batch_size=args.batch_size, shuffle=False,
                                       num_workers=args.num_workers, transform=self.test_transform)

        # Network
        self.model = Pose_Network(args).to(self.device)
        macs, params = get_model_complexity_info(self.model, (3, args.img_size, args.img_size), as_strings=True,
                                                 print_per_layer_stat=False, verbose=False)
        self.logger.info('{:<30}  {:<8}'.format('Computational complexity: ', macs))
        self.logger.info('{:<30}  {:<8}'.format('Number of parameters: ', params))

        # Loss
        self.criterion = nn.CrossEntropyLoss()
        
        # Optimizer & Scheduler
        self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr, weight_decay=args.weight_decay)
        
        iter_per_epoch = len(self.train_loader)
        self.warmup_scheduler = WarmUpLR(self.optimizer, iter_per_epoch * args.warm_epoch)

        if args.scheduler == 'cos':
            tmax = args.tmax # half-cycle 
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max = tmax, eta_min=args.min_lr, verbose=True)
        elif args.scheduler == 'cycle':
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)

        
        if args.multi_gpu:
            self.model = nn.DataParallel(self.model).to(self.device)

        # Train / Validate
        best_loss = np.inf
        best_acc = 0
        best_epoch = 0
        early_stopping = 0
        start = time.time()
        for epoch in range(1, args.epochs+1):
            self.epoch = epoch

            if args.scheduler == 'cos':
                if epoch > args.warm_epoch:
                    self.scheduler.step()

            # Training
            train_loss, train_acc = self.training(args)

            # Model weight in Multi_GPU or Single GPU
            state_dict= self.model.module.state_dict() if args.multi_gpu else self.model.state_dict()

            # Validation
            val_loss, val_acc = self.validate()

            # Save models
            if val_loss < best_loss:
                early_stopping = 0
                best_epoch = epoch
                best_loss = val_loss
                best_acc = val_acc

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),
                    }, os.path.join(save_path, 'best_model.pth'))
                self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
            else:
                early_stopping += 1

            # Early Stopping
            if early_stopping == args.patience:
                break

        self.logger.info(f'\nBest Val Epoch:{best_epoch} | Val Loss:{best_loss:.4f} | Val Acc:{best_acc:.4f}')
        end = time.time()
        self.logger.info(f'Total Process time:{(end - start) / 60:.3f}Minute')


    # Training
    def training(self, args):
        self.model.train()
        train_loss = AvgMeter()
        train_acc = 0

        scaler = grad_scaler.GradScaler()
        for i, (images, targets) in enumerate(tqdm(self.train_loader)):
            images = torch.tensor(images, device=self.device, dtype=torch.float32)
            targets = torch.tensor(targets, device=self.device, dtype=torch.long)
            
            if self.epoch <= args.warm_epoch:
                self.warmup_scheduler.step()

            self.model.zero_grad(set_to_none=True)
            if args.amp:
                with autocast():
                    preds = self.model(images)
                    loss = self.criterion(preds, targets)
                scaler.scale(loss).backward()

                # Gradient Clipping
                if args.clipping is not None:
                    scaler.unscale_(self.optimizer)
                    nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)

                scaler.step(self.optimizer)
                scaler.update()

            else:
                preds = self.model(images)
                loss = self.criterion(preds, targets)
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)
                self.optimizer.step()

            if args.scheduler == 'cycle':
                if self.epoch > args.warm_epoch:
                    self.scheduler.step()

            # Metric
            train_acc += (preds.argmax(dim=1) == targets).sum().item()
            # log
            train_loss.update(loss.item(), n=images.size(0))
            
        train_acc /= len(self.train_loader.dataset)

        self.logger.info(f'Epoch:[{self.epoch:03d}/{args.epochs:03d}]')
        self.logger.info(f'Train Loss:{train_loss.avg:.3f} | Acc:{train_acc:.4f}')
        return train_loss.avg, train_acc
            
    # Validation or Dev
    def validate(self):
        self.model.eval()
        with torch.no_grad():
            val_loss = AvgMeter()
            val_acc = 0

            for _, (images, targets) in enumerate(self.val_loader):
                images = torch.tensor(images, device=self.device, dtype=torch.float32)
                targets = torch.tensor(targets, device=self.device, dtype=torch.long)

                preds = self.model(images)
                loss = self.criterion(preds, targets)

                # Metric
                val_acc += (preds.argmax(dim=1) == targets).sum().item()
                # log
                val_loss.update(loss.item(), n=images.size(0))
            val_acc /= len(self.val_loader.dataset)

            self.logger.info(f'Valid Loss:{val_loss.avg:.3f} | Acc:{val_acc:.4f}')
        return val_loss.avg, val_acc

In [ ]:
## Case1
img = Image.open('./etc/숫자1_검지흔들기.png')
plt.figure(figsize=(10,5))
plt.imshow(img)

In [ ]:
# 시각화를 위한 Function
def visualize(folder_num):
    path = f'../data/train/{folder_num}/*.png'
    image_list = glob(path)
    length = len(image_list)
    
    fig, ax = plt.subplots(1, length, figsize=(50,10))
    for i, image in enumerate(image_list):
        image = Image.open(image).convert('RGB')
        ax[i].imshow(image)
    plt.show()

In [ ]:
df = pd.read_csv(opj(args.data_path, 'df_train.csv'))
df['groups'] = df['train_path'].apply(lambda x:x.split('/')[3])
df = df.drop_duplicates('groups')

In [ ]:
path = f'../data/train/'
number1_folder = df[df['answer_name'] == '숫자1']['groups'].tolist()
shake_folder = df[df['answer_name'] == '부정(검지 흔들기)']['groups'].tolist()

image1 = Image.open(opj(path, number1_folder[0], '1.png'))   # 352번 폴더
image2 = Image.open(opj(path, shake_folder[11], '1.png'))    # 489번 폴더
print(number1_folder[0], shake_folder[11])

fig, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(image1)
ax[1].imshow(image2)
plt.show()